In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install wandb
!pip install datasets
!pip install sentencepiece

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-oz8morwl
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-oz8morwl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 51.1 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2666998 sha256=e538e38fbdf703a1974496a76ccb7ce13c5174f859804c07f3cb13add996fc3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-o9g4sfwd/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [2]:
!mkdir 'data'
!mkdir 'output'

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

#zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/models/roberta-large (large-manual).zip", 'r')
#zip_ref.extractall("/output")
#zip_ref.close()

!unzip -q "drive/MyDrive/Colab Notebooks/models/roberta-large (large-manual).zip" -d output 

KeyboardInterrupt: ignored

In [4]:
import wandb
wandb.login() #6373b880383af25c4fd46232924ec59a0cf9456c

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from transformers import BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW, DistilBertForSequenceClassification, RobertaForSequenceClassification
from datasets import load_dataset, load_metric, Features, ClassLabel

import torch
import gc
import json
import pandas as pd
import numpy as np

labels = ClassLabel(num_classes=2, names=['NOTCLAIM', 'CLAIM'])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(predictions)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(examples):
    str_2_int = { "NOTCLAIM": 0, "CLAIM": 1 }
    tokenized_batch = tokenizer(examples['sentence'], padding="max_length", truncation=True)        
    tokenized_batch['label'] = [str_2_int[label] for label in examples['label']]  
    return tokenized_batch

metric = load_metric("accuracy")


#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

#model = RobertaForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=2)


#tokenizer = AutoTokenizer.from_pretrained("roberta-large")

#model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)

dataset = load_dataset('json', data_files={
    'train': 'data/train.json',
    'test': 'data/test.json'
})

train_dataset = dataset['train']

test_dataset = dataset['test']

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(['sentence_text', 'tokens', 'pos'])
test_dataset = test_dataset.remove_columns(['sentence_text', 'tokens', 'pos'])

training_args = TrainingArguments(
    output_dir='output',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='logs',            # directory for storing logs
    save_steps=1000,
    logging_steps=100,
    learning_rate=5e-6,
    do_eval=True
)

gc.collect()
torch.cuda.empty_cache()

trainer = Trainer(
    model=model, train_dataset=train_dataset, args=training_args, eval_dataset=test_dataset, compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model('output')

trainer.evaluate()


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f74cd5106d0>> (for pre_run_cell):


Exception: ignored

https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplalg3i42


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/81e970e5e6ec68be12da0f8f3b2f2469c78d579282299a2ea65b4b7441719107.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/81e970e5e6ec68be12da0f8f3b2f2469c78d579282299a2ea65b4b7441719107.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/distilbert-base-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpo1xrx4fz


...

  0%|          | 0/2 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running training *****
  Num examples = 3492
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 17460
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f74cd5106d0>> (for post_run_cell):


Exception: ignored

In [ ]:
from google.colab import auth
auth.authenticate_user()

path = "distilbert-base-uncased_large-manual_80.3"

!gsutil cp output/checkpoint-2000/* gs://comp66090-storage/claimExtraction/models/AIDA_hyperlinks/{path}/

Copying file://output/checkpoint-2000/config.json [Content-Type=application/json]...
Copying file://output/checkpoint-2000/optimizer.pt [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://output/checkpoint-2000/pytorch_model.bin [Content-Type=application/octet-stream]...
Copying file://output/checkpoint-2000/rng_stat

In [ ]:
#model = TheModelClass(*args, **kwargs)

#model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

wandb.init()

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

#model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

model.load_state_dict(torch.load('output/checkpoint-1000/pytorch_model.bin'))

trainer = Trainer(
    model=model, train_dataset=train_dataset, args=training_args, eval_dataset=test_dataset, compute_metrics=compute_metrics
)

trainer.evaluate()

eval/loss,0.8198
eval/accuracy,0.80387
eval/runtime,6.9501
eval/samples_per_second,52.086
eval/steps_per_second,13.093
train/global_step,0
_runtime,12
_timestamp,1628810663
_step,0


eval/loss,▁
eval/accuracy,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
_runtime,▁
_timestamp,▁
_step,▁


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.10.0.dev0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


[1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1
 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0
 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1
 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 0
 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 0
 0 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1
 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 0
 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 1 1]


{'eval_accuracy': 0.7900552486187845,
 'eval_loss': 0.5359231233596802,
 'eval_runtime': 6.5441,
 'eval_samples_per_second': 55.317,
 'eval_steps_per_second': 13.906}

In [ ]:
#wandb.init()

#from transformers import BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW, DistilBertForSequenceClassification, RobertaForSequenceClassification
#from datasets import load_dataset, load_metric, Features

#import torch

#import numpy as np

#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    print(predictions)
#    return metric.compute(predictions=predictions, references=labels)

#def tokenize_function(examples):
#    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

#metric = load_metric("accuracy")

#tokenizer = AutoTokenizer.from_pretrained("roberta-large")

#dataset = load_dataset('csv', data_files={
#    "train": 'data/train_rev.csv',
#    "test": 'data/test_rev.csv'
#})

#train_dataset = dataset['train']

#test_dataset = dataset['test']

#train_dataset = train_dataset.map(tokenize_function, batched=True)

#test_dataset = test_dataset.map(tokenize_function, batched=True)

model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

model.load_state_dict(torch.load('output/checkpoint-3500/pytorch_model.bin'))

training_args = TrainingArguments(
    output_dir='output',              # output directory
    num_train_epochs=5,               # total number of training epochs
    per_device_train_batch_size=4,    # batch size per device during training
    per_device_eval_batch_size=4,     # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='logs',               # directory for storing logs
    save_steps=1000,
    logging_steps=100,
    learning_rate=5e-6,
    do_eval=True
)

trainer = Trainer(
    model=model, train_dataset=train_dataset, args=training_args, eval_dataset=test_dataset, compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Using custom data configuration default-fbdae53a96d0d6df
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-fbdae53a96d0d6df/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-fbdae53a96d0d6df/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-4a6a1e469cb5722f.arrow


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

Step,Training Loss
100,0.432500
200,0.473400
300,0.415700
400,0.552400
500,0.625700
600,0.553000
700,0.559000
800,0.500600
900,0.320700
1000,0.440300


Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-3000
Configuration saved in output/checkpoint-3000/config.json
Model weights saved in output/checkpoint-3000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-4000
Configuration saved in output/checkpoint-4000/config.json
Model weights saved in output/checkpoint-4000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 1164
  Batch size 

NameError: ignored

In [ ]:
!printf 'y\ny\ny\n' | python run_glue.py --model_name_or_path roberta-large \
                    --train_file data/train.csv \
                    --validation_file data/dev.csv \
                    --test_file data/test.csv \
                    --do_train \
                    --do_eval \
                    --do_predict \
                    --max_seq_length 128 \
                    --per_gpu_train_batch_size 2 \
                    --learning_rate 4e-4 \
                    --num_train_epochs 3 \
                    --output_dir output \
                    --overwrite_output_dir \
                    --logging_steps 50

2021-08-12 14:46:13.434691: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/12/2021 14:46:15 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/12/2021 14:46:15 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=0.0004,
leng

In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW, DistilBertForSequenceClassification
from datasets import load_dataset, load_metric, Features

import torch
import gc

import json

import pandas as pd

import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(predictions)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

metric = load_metric("accuracy")

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")


model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

model.load_state_dict(torch.load('output/checkpoint-2500/pytorch_model.bin'))

dataset = load_dataset('csv', data_files={
    "train": 'data/train_rev.csv',
    "test": 'data/test_rev.csv'
})

train_dataset = dataset['train']

test_dataset = dataset['test']

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)

wandb.init()

trainer = Trainer(
    model=model, train_dataset=train_dataset, eval_dataset=test_dataset, compute_metrics=compute_metrics
)

trainer.evaluate()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3d7cdcc5f968db31/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


wandb: Currently logged in as: jmshaker (use `wandb login --relogin` to force relogin)


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 1164
  Batch size = 8


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import csv

predLabels = []

predict = open("output/predict_results_None.txt", "r")

lines = predict.readlines()

for line in lines[1:]:
    predLabels.append(line.split()[1])

realLabels = []

with open('data/test.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        realLabels.append(row['label'].strip())

correctPreds = 0

for i in range(0, len(realLabels)):
    if (realLabels[i] == predLabels[i]):
        correctPreds = correctPreds + 1

accuracy = (correctPreds / len(realLabels)) * 100

print("Accuracy = ", str(accuracy) + "%")

Accuracy =  49.140893470790374%


In [ ]:
import shutil

shutil.rmtree('drive')

KeyboardInterrupt: ignored

In [ ]:
!unzip datasets

Archive:  datasets.zip
  inflating: datasets/__init__.py    
   creating: datasets/__pycache__/
  inflating: datasets/__pycache__/__init__.cpython-38.pyc  
  inflating: datasets/__pycache__/arrow_dataset.cpython-38.pyc  
  inflating: datasets/__pycache__/arrow_reader.cpython-38.pyc  
  inflating: datasets/__pycache__/arrow_writer.cpython-38.pyc  
  inflating: datasets/__pycache__/builder.cpython-38.pyc  
  inflating: datasets/__pycache__/config.cpython-38.pyc  
  inflating: datasets/__pycache__/dataset_dict.cpython-38.pyc  
  inflating: datasets/__pycache__/features.cpython-38.pyc  
  inflating: datasets/__pycache__/fingerprint.cpython-38.pyc  
  inflating: datasets/__pycache__/hf_api.cpython-38.pyc  
  inflating: datasets/__pycache__/info.cpython-38.pyc  
  inflating: datasets/__pycache__/inspect.cpython-38.pyc  
  inflating: datasets/__pycache__/keyhash.cpython-38.pyc  
  inflating: datasets/__pycache__/load.cpython-38.pyc  
  inflating: datasets/__pycache__/metric.cpython-38.pyc  
 